In [ ]:
from pandas import read_csv
import pandas as pd

# Specify the encoding as detected ('ascii')
train_data = pd.read_csv('/content/srsstat_train_data.csv', encoding='ascii')  # Updated encoding to 'ascii'

test_data = pd.read_csv('/content/srsstat_test_data.csv', encoding='ascii')  # Updated encoding to 'ascii'

In [ ]:
train_data.value_counts('cluster')

,count
cluster,
1,7202
0,6913
2,6885


In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21000 entries, 0 to 20999
Data columns (total 87 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customer_id                21000 non-null  int64  
 1   age                        21000 non-null  int64  
 2   gender                     21000 non-null  object 
 3   income_bracket             21000 non-null  object 
 4   loyalty_program            21000 non-null  object 
 5   membership_years           21000 non-null  int64  
 6   churned                    21000 non-null  object 
 7   marital_status             21000 non-null  object 
 8   number_of_children         21000 non-null  int64  
 9   education_level            21000 non-null  object 
 10  occupation                 21000 non-null  object 
 11  transaction_id             21000 non-null  int64  
 12  product_id                 21000 non-null  int64  
 13  product_category           21000 non-null  obj

## KNN Without Balancing - All Features

In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from collections import Counter
from scipy.stats import randint
import pandas as pd
import numpy as np

# Verify train_data and test_data exist from previous steps
if 'train_data' not in globals() or 'test_data' not in globals():
    raise ValueError("train_data or test_data not found! Ensure previous steps are executed.")

# Define the attributes as specified
selected_features = [
    # Categorical attributes
    'last_purchase_month', 'promotion_end_month', 'product_manufacture_month',
    'month_of_year', 'product_expiry_date_year', 'product_manufacture_year',
    'transaction_year', 'product_expiry_date_month', 'transaction_month',
    'high_value_purchase', 'week_of_year', 'promotion_start_month', 'day_of_week',
    'purchase_frequency', 'customer_city', 'gender', 'weekend', 'store_state',
    'email_subscriptions', 'store_location', 'high_value_quantity',
    # Numeric attributes
    'customer_support_calls', 'product_review_count', 'days_since_last_purchase',
    'online_purchases', 'distance_to_store', 'product_rating', 'total_transactions',
    'product_weight', 'total_items_purchased', 'unit_price', 'total_returned_value',
    'membership_years', 'discount_applied', 'avg_discount_used', 'product_shelf_life',
    'total_returned_items', 'transaction_hour', 'min_single_purchase_value',
    'number_of_children', 'product_stock', 'avg_purchase_value',
    'avg_items_per_transaction', 'website_visits', 'age', 'max_single_purchase_value',
    'avg_spent_per_category', 'total_discounts_received', 'product_return_rate',
    'avg_transaction_value', 'in_store_purchases'
]

# Filter features that exist in train_data
selected_features = [col for col in selected_features if col in train_data.columns]
print("Selected features for KNN:", selected_features)

# Define features (X) and target (y) for train and test sets
X_train = train_data[selected_features]
y_train = train_data['cluster']
X_test = test_data[selected_features]
y_test = test_data['cluster']

# Print class distribution in training set
print("Class Distribution in Training Set:", Counter(y_train))

# Identify categorical and numeric columns among selected features
categorical_cols = [col for col in selected_features if col in ['last_purchase_month', 'promotion_end_month', 'product_manufacture_month', 'month_of_year', 'product_expiry_date_year',
                                                                'product_manufacture_year', 'transaction_year', 'product_expiry_date_month', 'transaction_month', 'high_value_purchase',
                                                                'week_of_year', 'promotion_start_month', 'day_of_week', 'purchase_frequency', 'customer_city', 'gender', 'weekend', 'store_state',
                                                                'email_subscriptions', 'store_location', 'high_value_quantity']]
numeric_cols = [col for col in selected_features if col not in categorical_cols]

# Preprocessing pipeline for encoding and scaling
preprocessor = ColumnTransformer(transformers=[
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ]), numeric_cols),
    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False))
    ]), categorical_cols)
])

# Create pipeline with preprocessing and KNN
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', KNeighborsClassifier())
])

# Define parameter distribution for RandomizedSearchCV
param_dist = {
    'classifier__n_neighbors': randint(3, 20),
    'classifier__weights': ['uniform', 'distance'],
    'classifier__metric': ['euclidean', 'manhattan', 'minkowski']
}

# Perform RandomizedSearchCV for hyperparameter tuning
random_search = RandomizedSearchCV(pipeline, param_distributions=param_dist, n_iter=10, cv=5,
                                   scoring='accuracy', n_jobs=-1, random_state=42)
random_search.fit(X_train, y_train)

# Best hyperparameters
best_params = random_search.best_params_
print(f"Best Hyperparameters: {best_params}")

# Train KNN with the best parameters
best_model = random_search.best_estimator_
best_model.fit(X_train, y_train)

# Predictions on test set
y_pred = best_model.predict(X_test)

# Get unique cluster labels for classification report
unique_clusters = sorted(y_train.unique())
target_names = [f'Cluster {i}' for i in unique_clusters]

# Classification Report and Metrics
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=target_names))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)

# Accuracy, Recall, Precision
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')

# Calculate Specificity for each cluster
def calculate_specificity(class_idx):
    tn = cm.sum() - cm[:, class_idx].sum() - cm[class_idx, :].sum() + cm[class_idx, class_idx]
    fp = cm[:, class_idx].sum() - cm[class_idx, class_idx]
    return tn / (tn + fp) if (tn + fp) > 0 else 0

# Calculate specificity for each cluster
specificity_scores = {f'Cluster {i}': calculate_specificity(idx) for idx, i in enumerate(unique_clusters)}

# Print metrics
print(f"\nAccuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
for cluster, spec in specificity_scores.items():
    print(f"Specificity for {cluster}: {spec:.4f}")

print("-----------------------------------------------")

# --- Cross-Validation on Full Dataset ---
# Combine train and test data for cross-validation
X_full = pd.concat([X_train, X_test], axis=0)
y_full = pd.concat([y_train, y_test], axis=0)

# Perform 5-fold cross-validation using the best model
cv_scores = cross_val_score(best_model, X_full, y_full, cv=5, scoring='accuracy', n_jobs=-1)
print(f"\nCross-Validation Results (5-fold):")
print(f"Mean Accuracy: {cv_scores.mean():.4f}")
print(f"Standard Deviation: {cv_scores.std():.4f}")
print(f"Individual Fold Scores: {cv_scores}")

Selected features for KNN: ['last_purchase_month', 'promotion_end_month', 'product_manufacture_month', 'month_of_year', 'product_expiry_date_year', 'product_manufacture_year', 'transaction_year', 'product_expiry_date_month', 'transaction_month', 'high_value_purchase', 'week_of_year', 'promotion_start_month', 'day_of_week', 'purchase_frequency', 'customer_city', 'gender', 'weekend', 'store_state', 'email_subscriptions', 'store_location', 'high_value_quantity', 'customer_support_calls', 'product_review_count', 'days_since_last_purchase', 'online_purchases', 'distance_to_store', 'product_rating', 'total_transactions', 'product_weight', 'total_items_purchased', 'unit_price', 'total_returned_value', 'membership_years', 'discount_applied', 'avg_discount_used', 'product_shelf_life', 'total_returned_items', 'transaction_hour', 'min_single_purchase_value', 'number_of_children', 'product_stock', 'avg_purchase_value', 'avg_items_per_transaction', 'website_visits', 'age', 'max_single_purchase_valu

## KNN - Without Balancing - Selected Features

In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from collections import Counter
from scipy.stats import randint
import pandas as pd
import numpy as np

# Verify train_data and test_data exist from previous steps
if 'train_data' not in globals() or 'test_data' not in globals():
    raise ValueError("train_data or test_data not found! Ensure previous steps are executed.")

# Define the attributes as specified
selected_features = [
    # Categorical attributes
    'last_purchase_month', 'promotion_end_month', 'product_manufacture_month',
    'month_of_year', 'product_expiry_date_year', 'product_manufacture_year',
    'transaction_year', 'product_expiry_date_month', 'transaction_month',
    'high_value_purchase', 'week_of_year', 'promotion_start_month', 'day_of_week',
    'purchase_frequency', 'customer_city', 'gender', 'weekend', 'store_state',
    'email_subscriptions', 'store_location', 'high_value_quantity',
    # Numeric attributes
    'customer_support_calls', 'product_review_count', 'days_since_last_purchase',
    'online_purchases', 'distance_to_store', 'product_rating', 'total_transactions',
    'product_weight', 'total_items_purchased', 'unit_price', 'total_returned_value',
    'membership_years', 'discount_applied', 'avg_discount_used', 'product_shelf_life',
    'total_returned_items', 'transaction_hour', 'min_single_purchase_value',
    'number_of_children', 'product_stock', 'avg_purchase_value',
    'avg_items_per_transaction', 'website_visits', 'age', 'max_single_purchase_value',
    'avg_spent_per_category', 'total_discounts_received', 'product_return_rate',
    'avg_transaction_value', 'in_store_purchases'
]

# Filter features that exist in train_data
selected_features = [col for col in selected_features if col in train_data.columns]
print("Initial selected features:", selected_features)

# Define features (X) and target (y) for train and test sets
X_train = train_data[selected_features]
y_train = train_data['cluster']
X_test = test_data[selected_features]
y_test = test_data['cluster']

# Print class distribution in training set
print("Class Distribution in Training Set:", Counter(y_train))

# Identify categorical and numeric columns among selected features
categorical_cols = [col for col in selected_features if col in ['last_purchase_month', 'promotion_end_month', 'product_manufacture_month', 'month_of_year', 'product_expiry_date_year',
                                                                'product_manufacture_year', 'transaction_year', 'product_expiry_date_month', 'transaction_month', 'high_value_purchase',
                                                                'week_of_year', 'promotion_start_month', 'day_of_week', 'purchase_frequency', 'customer_city', 'gender', 'weekend', 'store_state',
                                                                'email_subscriptions', 'store_location', 'high_value_quantity']]
numeric_cols = [col for col in selected_features if col not in categorical_cols]

# Preprocessing pipeline for encoding and scaling
preprocessor = ColumnTransformer(transformers=[
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ]), numeric_cols),
    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False))
    ]), categorical_cols)
])

# Preprocess the data to get transformed features
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

# Get transformed feature names
feature_names = preprocessor.get_feature_names_out()

# Convert transformed data to DataFrame for feature selection
X_train_transformed_df = pd.DataFrame(X_train_transformed, columns=feature_names)
X_test_transformed_df = pd.DataFrame(X_test_transformed, columns=feature_names)

# Feature selection using Random Forest to get feature importance
base_model = RandomForestClassifier(n_estimators=50, random_state=42)
base_model.fit(X_train_transformed_df, y_train)
feature_importances = base_model.feature_importances_
top_features_idx = np.argsort(feature_importances)[-10:]  # Get the top 10 features
top_10_features = feature_names[top_features_idx]
print(f"Top 10 Features: {list(top_10_features)}")

# Subset data with selected features
X_train_selected = X_train_transformed_df[top_10_features]
X_test_selected = X_test_transformed_df[top_10_features]

# Create pipeline with KNN (no preprocessing needed since data is pre-transformed)
pipeline = Pipeline([
    ('classifier', KNeighborsClassifier())
])

# Define parameter distribution for RandomizedSearchCV
param_dist = {
    'classifier__n_neighbors': randint(3, 20),
    'classifier__weights': ['uniform', 'distance'],
    'classifier__metric': ['euclidean', 'manhattan', 'minkowski']
}

# Perform RandomizedSearchCV for hyperparameter tuning
random_search = RandomizedSearchCV(pipeline, param_distributions=param_dist, n_iter=10, cv=5,
                                   scoring='accuracy', n_jobs=-1, random_state=42)
random_search.fit(X_train_selected, y_train)

# Best hyperparameters
best_params = random_search.best_params_
print(f"Best Hyperparameters: {best_params}")

# Train KNN with the best parameters
best_model = random_search.best_estimator_
best_model.fit(X_train_selected, y_train)

# Predictions on test set
y_pred = best_model.predict(X_test_selected)

# Get unique cluster labels for classification report
unique_clusters = sorted(y_train.unique())
target_names = [f'Cluster {i}' for i in unique_clusters]

# Classification Report and Metrics
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=target_names))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)

# Accuracy, Recall, Precision
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')

# Calculate Specificity for each cluster
def calculate_specificity(class_idx):
    tn = cm.sum() - cm[:, class_idx].sum() - cm[class_idx, :].sum() + cm[class_idx, class_idx]
    fp = cm[:, class_idx].sum() - cm[class_idx, class_idx]
    return tn / (tn + fp) if (tn + fp) > 0 else 0

# Calculate specificity for each cluster
specificity_scores = {f'Cluster {i}': calculate_specificity(idx) for idx, i in enumerate(unique_clusters)}

# Print metrics
print(f"\nAccuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
for cluster, spec in specificity_scores.items():
    print(f"Specificity for {cluster}: {spec:.4f}")

print("-----------------------------------------------")

# --- Cross-Validation on Full Dataset ---
# Combine train and test data for cross-validation
X_full = pd.concat([X_train_selected, X_test_selected], axis=0)
y_full = pd.concat([y_train, y_test], axis=0)

# Perform 5-fold cross-validation using the best model
cv_scores = cross_val_score(best_model, X_full, y_full, cv=5, scoring='accuracy', n_jobs=-1)
print(f"\nCross-Validation Results (5-fold):")
print(f"Mean Accuracy: {cv_scores.mean():.4f}")
print(f"Standard Deviation: {cv_scores.std():.4f}")
print(f"Individual Fold Scores: {cv_scores}")


Initial selected features: ['last_purchase_month', 'promotion_end_month', 'product_manufacture_month', 'month_of_year', 'product_expiry_date_year', 'product_manufacture_year', 'transaction_year', 'product_expiry_date_month', 'transaction_month', 'high_value_purchase', 'week_of_year', 'promotion_start_month', 'day_of_week', 'purchase_frequency', 'customer_city', 'gender', 'weekend', 'store_state', 'email_subscriptions', 'store_location', 'high_value_quantity', 'customer_support_calls', 'product_review_count', 'days_since_last_purchase', 'online_purchases', 'distance_to_store', 'product_rating', 'total_transactions', 'product_weight', 'total_items_purchased', 'unit_price', 'total_returned_value', 'membership_years', 'discount_applied', 'avg_discount_used', 'product_shelf_life', 'total_returned_items', 'transaction_hour', 'min_single_purchase_value', 'number_of_children', 'product_stock', 'avg_purchase_value', 'avg_items_per_transaction', 'website_visits', 'age', 'max_single_purchase_valu

## KNN - With Balancing - All Features

In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline as SKPipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline
from collections import Counter
from scipy.stats import randint
import pandas as pd
import numpy as np

# Verify train_data and test_data exist from previous steps
if 'train_data' not in globals() or 'test_data' not in globals():
    raise ValueError("train_data or test_data not found! Ensure previous steps are executed.")

# Define the attributes as specified
selected_features = [
    # Categorical attributes
    'last_purchase_month', 'promotion_end_month', 'product_manufacture_month',
    'month_of_year', 'product_expiry_date_year', 'product_manufacture_year',
    'transaction_year', 'product_expiry_date_month', 'transaction_month',
    'high_value_purchase', 'week_of_year', 'promotion_start_month', 'day_of_week',
    'purchase_frequency', 'customer_city', 'gender', 'weekend', 'store_state',
    'email_subscriptions', 'store_location', 'high_value_quantity',
    # Numeric attributes
    'customer_support_calls', 'product_review_count', 'days_since_last_purchase',
    'online_purchases', 'distance_to_store', 'product_rating', 'total_transactions',
    'product_weight', 'total_items_purchased', 'unit_price', 'total_returned_value',
    'membership_years', 'discount_applied', 'avg_discount_used', 'product_shelf_life',
    'total_returned_items', 'transaction_hour', 'min_single_purchase_value',
    'number_of_children', 'product_stock', 'avg_purchase_value',
    'avg_items_per_transaction', 'website_visits', 'age', 'max_single_purchase_value',
    'avg_spent_per_category', 'total_discounts_received', 'product_return_rate',
    'avg_transaction_value', 'in_store_purchases'
]

# Filter features that exist in train_data
selected_features = [col for col in selected_features if col in train_data.columns]
print("Initial selected features:", selected_features)

# Combine train_data and test_data for balancing
data = pd.concat([train_data, test_data], axis=0)
X = data[selected_features]
y = data['cluster']

# Print class distribution before balancing
print("Class Distribution Before Balancing:", Counter(y))

# Identify categorical and numeric columns among selected features
categorical_cols = [col for col in selected_features if col in ['last_purchase_month', 'promotion_end_month', 'product_manufacture_month', 'month_of_year', 'product_expiry_date_year',
                                                                'product_manufacture_year', 'transaction_year', 'product_expiry_date_month', 'transaction_month', 'high_value_purchase',
                                                                'week_of_year', 'promotion_start_month', 'day_of_week', 'purchase_frequency', 'customer_city', 'gender', 'weekend', 'store_state',
                                                                'email_subscriptions', 'store_location', 'high_value_quantity']]
numeric_cols = [col for col in selected_features if col not in categorical_cols]

# Preprocessing pipeline for encoding and scaling
preprocessor = ColumnTransformer(transformers=[
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ]), numeric_cols),
    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False))
    ]), categorical_cols)
])

# Preprocess the data before balancing to avoid string-to-float issues
X_transformed = preprocessor.fit_transform(X)
feature_names = preprocessor.get_feature_names_out()

# Define balancing strategy: target 2,000 samples per cluster
sampling_strategy_under = {1: 10000}
sampling_strategy_over = {0: 10000, 2: 10000}

# Create a pipeline for undersampling and oversampling
balancing_pipeline = ImbPipeline([
    ('undersample', RandomUnderSampler(sampling_strategy=sampling_strategy_under, random_state=42)),
    ('oversample', SMOTE(sampling_strategy=sampling_strategy_over, random_state=42))
])

# Apply balancing
X_resampled, y_resampled = balancing_pipeline.fit_resample(X_transformed, y)

# Check dataset distribution after balancing
print("After Balancing:", Counter(y_resampled))

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

# Create pipeline with KNN (no preprocessing needed since data is pre-transformed)
pipeline = SKPipeline([
    ('classifier', KNeighborsClassifier())
])

# Define parameter distribution for RandomizedSearchCV
param_dist = {
    'classifier__n_neighbors': randint(3, 20),
    'classifier__weights': ['uniform', 'distance'],
    'classifier__metric': ['euclidean', 'manhattan', 'minkowski']
}

# Perform RandomizedSearchCV for hyperparameter tuning
random_search = RandomizedSearchCV(pipeline, param_distributions=param_dist, n_iter=10, cv=5,
                                   scoring='accuracy', n_jobs=-1, random_state=42)
random_search.fit(X_train, y_train)

# Best hyperparameters
best_params = random_search.best_params_
print(f"Best Hyperparameters: {best_params}")

# Train KNN with the best parameters
best_model = random_search.best_estimator_
best_model.fit(X_train, y_train)

# Predictions on test set
y_pred = best_model.predict(X_test)

# Get unique cluster labels for classification report
unique_clusters = sorted(np.unique(y_train))
target_names = [f'Cluster {i}' for i in unique_clusters]

# Classification Report and Metrics
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=target_names))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)

# Accuracy, Recall, Precision
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')

# Calculate Specificity for each cluster
def calculate_specificity(class_idx):
    tn = cm.sum() - cm[:, class_idx].sum() - cm[class_idx, :].sum() + cm[class_idx, class_idx]
    fp = cm[:, class_idx].sum() - cm[class_idx, class_idx]
    return tn / (tn + fp) if (tn + fp) > 0 else 0

# Calculate specificity for each cluster
specificity_scores = {f'Cluster {i}': calculate_specificity(idx) for idx, i in enumerate(unique_clusters)}

# Print metrics
print(f"\nAccuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
for cluster, spec in specificity_scores.items():
    print(f"Specificity for {cluster}: {spec:.4f}")

print("-----------------------------------------------")

# --- Cross-Validation on Full Dataset ---
# Combine train and test data for cross-validation
X_full = np.vstack([X_train, X_test])
y_full = np.concatenate([y_train, y_test])

# Perform 5-fold cross-validation using the best model
cv_scores = cross_val_score(best_model, X_full, y_full, cv=5, scoring='accuracy', n_jobs=-1)
print(f"\nCross-Validation Results (5-fold):")
print(f"Mean Accuracy: {cv_scores.mean():.4f}")
print(f"Standard Deviation: {cv_scores.std():.4f}")
print(f"Individual Fold Scores: {cv_scores}")

Initial selected features: ['last_purchase_month', 'promotion_end_month', 'product_manufacture_month', 'month_of_year', 'product_expiry_date_year', 'product_manufacture_year', 'transaction_year', 'product_expiry_date_month', 'transaction_month', 'high_value_purchase', 'week_of_year', 'promotion_start_month', 'day_of_week', 'purchase_frequency', 'customer_city', 'gender', 'weekend', 'store_state', 'email_subscriptions', 'store_location', 'high_value_quantity', 'customer_support_calls', 'product_review_count', 'days_since_last_purchase', 'online_purchases', 'distance_to_store', 'product_rating', 'total_transactions', 'product_weight', 'total_items_purchased', 'unit_price', 'total_returned_value', 'membership_years', 'discount_applied', 'avg_discount_used', 'product_shelf_life', 'total_returned_items', 'transaction_hour', 'min_single_purchase_value', 'number_of_children', 'product_stock', 'avg_purchase_value', 'avg_items_per_transaction', 'website_visits', 'age', 'max_single_purchase_valu

## KNN - With Balancing - Selected Features

In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline as SKPipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline
from collections import Counter
from scipy.stats import randint
import pandas as pd
import numpy as np

# Verify train_data and test_data exist from previous steps
if 'train_data' not in globals() or 'test_data' not in globals():
    raise ValueError("train_data or test_data not found! Ensure previous steps are executed.")

# Define the attributes as specified
selected_features = [
    # Categorical attributes
    'last_purchase_month', 'promotion_end_month', 'product_manufacture_month',
    'month_of_year', 'product_expiry_date_year', 'product_manufacture_year',
    'transaction_year', 'product_expiry_date_month', 'transaction_month',
    'high_value_purchase', 'week_of_year', 'promotion_start_month', 'day_of_week',
    'purchase_frequency', 'customer_city', 'gender', 'weekend', 'store_state',
    'email_subscriptions', 'store_location', 'high_value_quantity',
    # Numeric attributes
    'customer_support_calls', 'product_review_count', 'days_since_last_purchase',
    'online_purchases', 'distance_to_store', 'product_rating', 'total_transactions',
    'product_weight', 'total_items_purchased', 'unit_price', 'total_returned_value',
    'membership_years', 'discount_applied', 'avg_discount_used', 'product_shelf_life',
    'total_returned_items', 'transaction_hour', 'min_single_purchase_value',
    'number_of_children', 'product_stock', 'avg_purchase_value',
    'avg_items_per_transaction', 'website_visits', 'age', 'max_single_purchase_value',
    'avg_spent_per_category', 'total_discounts_received', 'product_return_rate',
    'avg_transaction_value', 'in_store_purchases'
]

# Filter features that exist in train_data
selected_features = [col for col in selected_features if col in train_data.columns]
print("Initial selected features:", selected_features)

# Combine train_data and test_data for balancing
data = pd.concat([train_data, test_data], axis=0)
X = data[selected_features]
y = data['cluster']

# Print class distribution before balancing
print("Class Distribution Before Balancing:", Counter(y))

# Identify categorical and numeric columns among selected features
categorical_cols = [col for col in selected_features if col in ['last_purchase_month', 'promotion_end_month', 'product_manufacture_month', 'month_of_year', 'product_expiry_date_year',
                                                                'product_manufacture_year', 'transaction_year', 'product_expiry_date_month', 'transaction_month', 'high_value_purchase',
                                                                'week_of_year', 'promotion_start_month', 'day_of_week', 'purchase_frequency', 'customer_city', 'gender', 'weekend', 'store_state',
                                                                'email_subscriptions', 'store_location', 'high_value_quantity']]
numeric_cols = [col for col in selected_features if col not in categorical_cols]

# Preprocessing pipeline for encoding and scaling
preprocessor = ColumnTransformer(transformers=[
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ]), numeric_cols),
    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False))
    ]), categorical_cols)
])

# Preprocess the data before balancing to avoid string-to-float issues
X_transformed = preprocessor.fit_transform(X)
feature_names = preprocessor.get_feature_names_out()

# Define balancing strategy: target 2,000 samples per cluster
sampling_strategy_under = {1: 10000}
sampling_strategy_over = {0: 10000, 2: 10000}

# Create a pipeline for undersampling and oversampling
balancing_pipeline = ImbPipeline([
    ('undersample', RandomUnderSampler(sampling_strategy=sampling_strategy_under, random_state=42)),
    ('oversample', SMOTE(sampling_strategy=sampling_strategy_over, random_state=42))
])

# Apply balancing
X_resampled, y_resampled = balancing_pipeline.fit_resample(X_transformed, y)

# Check dataset distribution after balancing
print("After Balancing:", Counter(y_resampled))

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

# Convert balanced data to DataFrame for feature selection
X_train_df = pd.DataFrame(X_train, columns=feature_names)
X_test_df = pd.DataFrame(X_test, columns=feature_names)

# Feature selection using Random Forest to get feature importance
base_model = RandomForestClassifier(n_estimators=50, random_state=42)
base_model.fit(X_train_df, y_train)
feature_importances = base_model.feature_importances_
top_features_idx = np.argsort(feature_importances)[-10:]  # Get the top 10 features
top_10_features = feature_names[top_features_idx]
print(f"Top 10 Features: {list(top_10_features)}")

# Subset data with selected features
X_train_selected = X_train_df[top_10_features]
X_test_selected = X_test_df[top_10_features]

# Create pipeline with KNN (no preprocessing needed since data is pre-transformed)
pipeline = SKPipeline([
    ('classifier', KNeighborsClassifier())
])

# Define parameter distribution for RandomizedSearchCV
param_dist = {
    'classifier__n_neighbors': randint(3, 20),
    'classifier__weights': ['uniform', 'distance'],
    'classifier__metric': ['euclidean', 'manhattan', 'minkowski']
}

# Perform RandomizedSearchCV for hyperparameter tuning
random_search = RandomizedSearchCV(pipeline, param_distributions=param_dist, n_iter=10, cv=5,
                                   scoring='accuracy', n_jobs=-1, random_state=42)
random_search.fit(X_train_selected, y_train)

# Best hyperparameters
best_params = random_search.best_params_
print(f"Best Hyperparameters: {best_params}")

# Train KNN with the best parameters
best_model = random_search.best_estimator_
best_model.fit(X_train_selected, y_train)

# Predictions on test set
y_pred = best_model.predict(X_test_selected)

# Get unique cluster labels for classification report
unique_clusters = sorted(np.unique(y_train))
target_names = [f'Cluster {i}' for i in unique_clusters]

# Classification Report and Metrics
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=target_names))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)

# Accuracy, Recall, Precision
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')

# Calculate Specificity for each cluster
def calculate_specificity(class_idx):
    tn = cm.sum() - cm[:, class_idx].sum() - cm[class_idx, :].sum() + cm[class_idx, class_idx]
    fp = cm[:, class_idx].sum() - cm[class_idx, class_idx]
    return tn / (tn + fp) if (tn + fp) > 0 else 0

# Calculate specificity for each cluster
specificity_scores = {f'Cluster {i}': calculate_specificity(idx) for idx, i in enumerate(unique_clusters)}

# Print metrics
print(f"\nAccuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
for cluster, spec in specificity_scores.items():
    print(f"Specificity for {cluster}: {spec:.4f}")

print("-----------------------------------------------")

# --- Cross-Validation on Full Dataset ---
# Combine train and test data for cross-validation
X_full = pd.concat([X_train_selected, X_test_selected], axis=0)
y_full = pd.concat([y_train, y_test], axis=0)

# Perform 5-fold cross-validation using the best model
cv_scores = cross_val_score(best_model, X_full, y_full, cv=5, scoring='accuracy', n_jobs=-1)
print(f"\nCross-Validation Results (5-fold):")
print(f"Mean Accuracy: {cv_scores.mean():.4f}")
print(f"Standard Deviation: {cv_scores.std():.4f}")
print(f"Individual Fold Scores: {cv_scores}")

Initial selected features: ['last_purchase_month', 'promotion_end_month', 'product_manufacture_month', 'month_of_year', 'product_expiry_date_year', 'product_manufacture_year', 'transaction_year', 'product_expiry_date_month', 'transaction_month', 'high_value_purchase', 'week_of_year', 'promotion_start_month', 'day_of_week', 'purchase_frequency', 'customer_city', 'gender', 'weekend', 'store_state', 'email_subscriptions', 'store_location', 'high_value_quantity', 'customer_support_calls', 'product_review_count', 'days_since_last_purchase', 'online_purchases', 'distance_to_store', 'product_rating', 'total_transactions', 'product_weight', 'total_items_purchased', 'unit_price', 'total_returned_value', 'membership_years', 'discount_applied', 'avg_discount_used', 'product_shelf_life', 'total_returned_items', 'transaction_hour', 'min_single_purchase_value', 'number_of_children', 'product_stock', 'avg_purchase_value', 'avg_items_per_transaction', 'website_visits', 'age', 'max_single_purchase_valu